# 配置

使用前请确保 HM3D 的标签和配置对应的 `tar` 文件已经解压完成，但如果场景对应的 `tar` 文件过大，则无需解压，直接运行

In [ ]:
settings = {
    "width": 256,
    "height": 256,
    'T_per_sample': 100, # 每个场景中，随机产生多少个位置
    'rotation_step': 360.0, # 在每个位置上，产生该值除 360 个样本，相邻两个样本间摄像机朝向相差该值；该值为 360 表示每个位置采一个样本

    # 数据集位置
    "dataset": "../../../data/hm3d_train/",
    "dataset_config": "../../../data/hm3d_train/hm3d_annotated_train_basis.scene_dataset_config.json", # Configuration of the dataset


    # 采样结果的储存位置
    "rgb_path": "../../../data/rgbx/train/rgb",
    "depth_path": "../../../data/rgbx/train/depth",
    "semantic_path": "../../../data/rgbx/train/label",

    "sensor_height": 1.5,  # 传感器的高度，单位：米
    "enable_physics": False, 
}

配置完成后运行剩下的全部代码，即可在 `settings['rgb_path'/'depth_path'/'semantic_path']` 下找到采样后的 RGB/D/Semantic 图像。

# 主体部分

## 可视化和存储

如果需要改变存储的格式，请随意修改 `save_sample()`

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import os
from tqdm.auto import tqdm

# function to display the topdown map
from PIL import Image

def display_sample(rgb_obs, semantic_obs=np.array([]), depth_obs=np.array([])):
    from habitat_sim.utils.common import d3_40_colors_rgb

    rgb_img = Image.fromarray(rgb_obs, mode="RGBA")

    arr = [rgb_img]
    titles = ["rgb"]
    if semantic_obs.size != 0:
        semantic_img = Image.new("P", (semantic_obs.shape[1], semantic_obs.shape[0]))
        semantic_img.putpalette(d3_40_colors_rgb.flatten())
        semantic_img.putdata((semantic_obs.flatten() % 40).astype(np.uint8))
        semantic_img = semantic_img.convert("RGBA")
        arr.append(semantic_img)
        titles.append("semantic")

    if depth_obs.size != 0:
        depth_img = Image.fromarray((depth_obs / 10 * 255).astype(np.uint8), mode="L")
        arr.append(depth_img)
        titles.append("depth")

    plt.figure(figsize=(12, 8))
    for i, data in enumerate(arr):
        ax = plt.subplot(1, 3, i + 1)
        ax.axis("off")
        ax.set_title(titles[i])
        plt.imshow(data)
    plt.show(block=False)

def save_sample(rgb_obs: np.ndarray, depth_obs: np.ndarray, semantic_obs: np.ndarray, rgb_path: str, depth_path: str, semantic_path: str):
    rgb_img = Image.fromarray(rgb_obs, mode="RGBA")
    rgb_img.save(rgb_path)

    np.save(depth_path, depth_obs)
    np.save(semantic_path, semantic_obs)

## Habitat-Sim 配置

In [ ]:
import habitat_sim
import sys
sys.path.append('../../..')

In [ ]:
def make_cfg(scene_dir, scene_name):
    sim_cfg = habitat_sim.SimulatorConfiguration()
    sim_cfg.gpu_device_id = 0
    sim_cfg.scene_id =  os.path.join(settings['dataset'], scene_dir, scene_name + '.basis.glb')
    sim_cfg.enable_physics = settings['enable_physics']
    sim_cfg.scene_dataset_config_file = settings['dataset_config']


    color_sensor_spec = habitat_sim.CameraSensorSpec()
    color_sensor_spec.uuid = 'color_sensor'
    color_sensor_spec.sensor_type = habitat_sim.SensorType.COLOR
    color_sensor_spec.resolution = [settings['height'], settings['width']]
    color_sensor_spec.position = [0.0, settings['sensor_height'], 0.0]
    color_sensor_spec.sensor_subtype = habitat_sim.SensorSubType.PINHOLE

    depth_sensor_spec = habitat_sim.CameraSensorSpec()
    depth_sensor_spec.uuid = 'depth_sensor'
    depth_sensor_spec.sensor_type = habitat_sim.SensorType.DEPTH
    depth_sensor_spec.position = [0.0, settings['sensor_height'], 0.0]
    depth_sensor_spec.resolution = [settings['height'], settings['width']]
    depth_sensor_spec.sensor_subtype = habitat_sim.SensorSubType.PINHOLE

    semantic_sensor_spec = habitat_sim.CameraSensorSpec()
    semantic_sensor_spec.uuid = 'semantic_sensor'
    semantic_sensor_spec.sensor_type = habitat_sim.SensorType.SEMANTIC
    semantic_sensor_spec.resolution = [settings["height"], settings["width"]]
    semantic_sensor_spec.position = [0.0, settings["sensor_height"], 0.0]
    semantic_sensor_spec.sensor_subtype = habitat_sim.SensorSubType.PINHOLE

    sensor_specs = [color_sensor_spec, depth_sensor_spec, semantic_sensor_spec]

    agent_cfg = habitat_sim.agent.AgentConfiguration()
    agent_cfg.sensor_specifications = sensor_specs
    agent_cfg.action_space = {
        "move_forward": habitat_sim.agent.ActionSpec(
            "move_forward", habitat_sim.agent.ActuationSpec(amount=0.25)
        ),
        "turn_left": habitat_sim.agent.ActionSpec(
            "turn_left", habitat_sim.agent.ActuationSpec(amount=settings['rotation_step'])
        ),
        "turn_right": habitat_sim.agent.ActionSpec(
            "turn_right", habitat_sim.agent.ActuationSpec(amount=settings['rotation_step'])
        ),
    }

    return habitat_sim.Configuration(sim_cfg, [agent_cfg])

## 在单独场景中采样

In [ ]:
from models.perception.utils import ClassReducer
def sample(scene_name, sim, T: int, rgb_path: str, depth_path: str, semantic_path: str, display: bool=False):
    import random
    import os
    reducer = ClassReducer.from_sim(sim)
    paths = {'color': rgb_path, 'depth': depth_path, 'semantic': semantic_path}
    for t in range(T):
        pos = None
        while pos is None or not sim.pathfinder.is_navigable(pos):
            pos = sim.pathfinder.get_random_navigable_point()
        rot = np.array(random.random() * 2 * np.pi)
        orientation = np.quaternion(np.cos(rot / 2), 0, -1 * np.sin(rot / 2), 0)
        # orientation = np.quaternion(1, 0, 0, 0)
        state = habitat_sim.AgentState(position=pos, rotation=orientation, sensor_states=sim.get_agent(0).get_state().sensor_states)
        sim.get_agent(0).set_state(state=state, reset_sensors=False)

        for k in range(int(360 / settings['rotation_step'])):
            obs = sim.step('turn_right')
            if display:
                display_sample(obs['color_sensor'], reducer.instance_to_category(obs['semantic_sensor']), obs['depth_sensor'])
            prefix = scene_name + "_" + f"{pos}".replace(' ','').replace('[', '').replace(']', '') + "_" + f"{k}"
            save_sample(
                obs['color_sensor'], obs['depth_sensor'], reducer.instance_to_category(obs['semantic_sensor']), 
                os.path.join(rgb_path, prefix + ".png"),
                os.path.join(depth_path, prefix + ".npy"),
                os.path.join(semantic_path, prefix + ".npy"),
                )

In [ ]:
from IPython.display import clear_output

def make_samples_for_scene(scene_dir:str, scene_name:str=None):
    if scene_name is None:
        scene_name = scene_dir[scene_dir.find('-') + 1:]
    cfg = make_cfg(scene_dir, scene_name)
    
    try:
        sim.close()
    except NameError:
        pass
    sim = habitat_sim.Simulator(cfg)
    sample(scene_name, sim, settings['T_per_sample'], settings['rgb_path'], settings['depth_path'], settings['semantic_path'])
    sim.close()
    clear_output(wait=True)

## 在所有场景中采样

In [ ]:
import tarfile
import utils
def make_samples():
    # 无论 main.tar 是否解压，semantic.tar 都会给出有语义标记的目录
    dirs = []
    for dir in os.listdir(settings['dataset']):
        if not os.path.isdir(os.path.join(settings['dataset'], dir)):
            continue

        contents = os.listdir(os.path.join(settings['dataset'], dir))
        found_semantic = False
        for c in contents:
            if 'semantic' in c:
                found_semantic = True
        if not found_semantic:
            continue
        dirs.append(dir)
    for dir in tqdm(dirs):
        print(dir)
        with utils.fulfill(settings['dataset'], dir):
            make_samples_for_scene(dir)

In [ ]:
make_samples()

In [ ]:
with tarfile.open(os.path.join(settings['dataset'], 'main.tar')) as tar:
    print([file for file in tar.getnames() if '00567' in file])